In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import gc
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
Input = "../input/"
# Any results you write to the current directory are saved as output.

In [ ]:
ori_data = pd.read_csv(Input + "train_V2.csv")
ori_test = pd.read_csv(Input + "test_V2.csv")


In [ ]:
print (ori_data.shape)

In [ ]:
print (ori_test.shape)

In [ ]:
# Memory saving function credit to https://www.kaggle.com/gemartin/load-data-reduce-memory-usage
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    for col in df.columns:
        col_type = df[col].dtype

        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
                    
    return df

In [ ]:
ori_data = reduce_mem_usage(ori_data)
ori_test = reduce_mem_usage(ori_test)

In [ ]:
# changed from https://www.kaggle.com/anycode/simple-nn-baseline
# Adding features based on https://www.kaggle.com/powercode/sklearn-mlp
def feature_engineering(is_train=True, df_train = ori_data, df_test = ori_test):
    if is_train: 
        print("processing train.csv")
        df = df_train
        df = df[df['maxPlace'] > 1]
    else:
        print("processing test.csv")
        df = df_test
    
    print ("Adding some features")
    
    df['totalDistance'] = df['rideDistance'] + df["walkDistance"] + df["swimDistance"]
    df["skill"] = df["headshotKills"] + df["roadKills"]
    df['healthitems'] = df['heals'] + df['boosts']
    
    '''
    df['killPlace_over_maxPlace'] = df['killPlace'] / df['maxPlace']
    df['headshotKills_over_kills'] = df['headshotKills'] / df['kills']
    df['distance_over_weapons'] = df['totalDistance'] / df['weaponsAcquired']
    df['walkDistance_over_heals'] = df['walkDistance'] / df['heals']
    df['walkDistance_over_kills'] = df['walkDistance'] / df['kills']
    df['killsPerWalkDistance'] = df['kills'] / df['walkDistance']
    df['headshotrate'] = df['kills']/df['headshotKills']
    df['killStreakrate'] = df['killStreaks']/df['kills']
    
    df[df == np.Inf] = np.NaN
    df[df == np.NINF] = np.NaN
    
    print("Removing Na's from df")
    df.fillna(0, inplace=True)
    '''
    # df = df[:100]
    
    print("remove some columns")
    target = 'winPlacePerc'
    features = list(df.columns)
    features.remove("Id")
    features.remove("matchId")
    features.remove("groupId")
    features.remove("matchType")
    
    # matchType = pd.get_dummies(df['matchType'])
    # df = df.join(matchType)    
    
    y = None
    
    print("get target")
    if is_train: 
        y = np.array(df.groupby(['matchId','groupId'])[target].agg('mean'), dtype=np.float64)
        features.remove(target)

    print("get group mean feature")
    agg = df.groupby(['matchId','groupId'])[features].agg('mean')
    agg_rank = agg.groupby('matchId')[features].rank(pct=True).reset_index()
    
    if is_train: df_out = agg.reset_index()[['matchId','groupId']]
    else: df_out = df[['matchId','groupId']]

    df_out = df_out.merge(agg.reset_index(), suffixes=["", ""], how='left', on=['matchId', 'groupId'])
    df_out = df_out.merge(agg_rank, suffixes=["_mean", "_mean_rank"], how='left', on=['matchId', 'groupId'])
    
    print("get group max feature")
    agg = df.groupby(['matchId','groupId'])[features].agg('max')
    agg_rank = agg.groupby('matchId')[features].rank(pct=True).reset_index()
    df_out = df_out.merge(agg.reset_index(), suffixes=["", ""], how='left', on=['matchId', 'groupId'])
    df_out = df_out.merge(agg_rank, suffixes=["_max", "_max_rank"], how='left', on=['matchId', 'groupId'])
    
    print("get group min feature")
    agg = df.groupby(['matchId','groupId'])[features].agg('min')
    agg_rank = agg.groupby('matchId')[features].rank(pct=True).reset_index()
    df_out = df_out.merge(agg.reset_index(), suffixes=["", ""], how='left', on=['matchId', 'groupId'])
    df_out = df_out.merge(agg_rank, suffixes=["_min", "_min_rank"], how='left', on=['matchId', 'groupId'])
    
    print("get group size feature")
    agg = df.groupby(['matchId','groupId']).size().reset_index(name='group_size')
    df_out = df_out.merge(agg, how='left', on=['matchId', 'groupId'])
    
    print("get match mean feature")
    agg = df.groupby(['matchId'])[features].agg('mean').reset_index()
    df_out = df_out.merge(agg, suffixes=["", "_match_mean"], how='left', on=['matchId'])
    
    print("get match size feature")
    agg = df.groupby(['matchId']).size().reset_index(name='match_size')
    df_out = df_out.merge(agg, how='left', on=['matchId'])
    
    df_out.drop(["matchId", "groupId"], axis=1, inplace=True)

    X = np.array(df_out, dtype=np.float64)
    
    feature_names = list(df_out.columns)

    del df, df_out, agg, agg_rank
    gc.collect()

    return X, y, feature_names

In [ ]:
X_train, y_train, feature_names = feature_engineering()
print (len(feature_names))
print (X_train.shape)
print (y_train.shape)

In [ ]:
print (feature_names)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization, Input, Activation
from keras.callbacks import LearningRateScheduler, EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers
from keras.models import load_model, Model
import keras


In [ ]:
'''
model = Sequential()
model.add(Dense(512, kernel_initializer='he_normal', input_dim=X_train.shape[1], activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.1))
model.add(Dense(256, kernel_initializer='he_normal', activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.1))
model.add(Dense(128, kernel_initializer='he_normal', activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.1))
model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
'''

main_input = Input(shape=(X_train.shape[1],), name='main_input')
x = Dense(1024, kernel_initializer='he_normal')(main_input)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.LeakyReLU(alpha=0.3)(x)
# got 0
x = Dense(512, kernel_initializer='he_normal')(x)
x = keras.layers.BatchNormalization()(x)
temp_1 = keras.layers.LeakyReLU(alpha=0.3)(x)
# temp_1 = x
# got 1
x = Dense(512, kernel_initializer='he_normal')(temp_1)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.LeakyReLU(alpha=0.3)(x)
# got 2
x = Dense(512, kernel_initializer='he_normal')(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.LeakyReLU(alpha=0.3)(x)
# got 3
x = Dense(512, kernel_initializer='he_normal')(x)
x = keras.layers.BatchNormalization()(x)
added_1 = keras.layers.Add()([temp_1,x])
x = keras.layers.LeakyReLU(alpha=0.3)(added_1)
# got 4
x = Dense(256, kernel_initializer='he_normal')(x)
x = keras.layers.BatchNormalization()(x)
temp_2 = keras.layers.LeakyReLU(alpha=0.3)(x)
# temp_2 = x
# got 5
x = Dense(256, kernel_initializer='he_normal')(temp_2)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.LeakyReLU(alpha=0.3)(x)
# got 6
x = Dense(256, kernel_initializer='he_normal')(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.LeakyReLU(alpha=0.3)(x)
# got 7
x = Dense(256, kernel_initializer='he_normal')(x)
x = keras.layers.BatchNormalization()(x)
added_2 = keras.layers.Add()([temp_2,x])
x = keras.layers.LeakyReLU(alpha=0.3)(added_2)
# got 8
x = Dense(128, kernel_initializer='he_normal')(x)
x = keras.layers.BatchNormalization()(x)
temp_3 = keras.layers.LeakyReLU(alpha=0.3)(x)
#temp_3 = x
# got 9
x = Dense(128, kernel_initializer='he_normal')(temp_3)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.LeakyReLU(alpha=0.3)(x)
# got 10
x = Dense(128, kernel_initializer='he_normal')(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.LeakyReLU(alpha=0.3)(x)
# got 11
x = Dense(128, kernel_initializer='he_normal')(x)
x = keras.layers.BatchNormalization()(x)
added_3 = keras.layers.Add()([temp_3,x])
x = keras.layers.LeakyReLU(alpha=0.3)(added_3)
# got 12
x = Dense(64, kernel_initializer='he_normal')(x)
x = keras.layers.BatchNormalization()(x)
temp_4 = keras.layers.LeakyReLU(alpha=0.3)(x)
# got 13
x = Dense(64, kernel_initializer='he_normal')(temp_4)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.LeakyReLU(alpha=0.3)(x)
# got 14
x = Dense(64, kernel_initializer='he_normal')(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.LeakyReLU(alpha=0.3)(x)
# got 15
x = Dense(64, kernel_initializer='he_normal')(x)
x = keras.layers.BatchNormalization()(x)
added_4 = keras.layers.Add()([temp_4,x])
x = keras.layers.LeakyReLU(alpha=0.3)(added_4)
# got 16
x = Dense(32, kernel_initializer='he_normal')(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.LeakyReLU(alpha=0.3)(x)
# got 17
x = Dense(16, kernel_initializer='he_normal')(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.LeakyReLU(alpha=0.3)(x)
# got 18
out = Dense(1, kernel_initializer='normal', activation='sigmoid')(x)
# got output
model = Model(inputs=main_input, outputs=out)


In [ ]:
print (X_train.shape)
print (y_train.shape)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

x_train, x_valid, label_train, label_valid = train_test_split(X_train, y_train, test_size = 0.2, random_state = 1234)
del X_train, y_train
gc.collect()
print ("Finishing Split!")
# scaler = preprocessing.MinMaxScaler(feature_range=(-1, 1)).fit(x_train)
scaler = preprocessing.QuantileTransformer().fit(x_train)

x_train = scaler.transform(x_train)
x_valid = scaler.transform(x_valid)
X_test, _, _ = feature_engineering(is_train = False)
X_test = scaler.transform(X_test)

print("x_train", x_train.shape, x_train.min(), x_train.max())
print("x_valid", x_valid.shape, x_valid.min(), x_valid.max())
print("X_test", X_test.shape, X_test.min(), X_test.max())


In [ ]:
optimizer = optimizers.Adam(lr=0.01, epsilon=1e-8, decay=1e-4, amsgrad=True)
# optimizer = optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=0.0)

model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])

def step_decay_schedule(initial_lr=1e-3, decay_factor=0.75, step_size=10, verbose=0):
    '''
    Wrapper function to create a LearningRateScheduler with step decay schedule.
    '''
    def schedule(epoch):
        return initial_lr * (decay_factor ** np.floor(epoch/step_size))
    
    return LearningRateScheduler(schedule, verbose)

lr_sched = step_decay_schedule(initial_lr=0.1, decay_factor=0.9, step_size=1, verbose=1)
# early_stopping = EarlyStopping(monitor='val_mean_absolute_error', min_delta=0, mode = 'min', patience=8, verbose=1)

history = model.fit(x_train, label_train, 
                 validation_data=(x_valid, label_valid),
                 epochs=50,
                 batch_size=32768,
                 callbacks=[lr_sched], 
                 verbose=1)


In [ ]:
# Plot training & validation loss values
import matplotlib.pyplot as plt
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Plot training & validation mae values
plt.plot(history.history['mean_absolute_error'])
plt.plot(history.history['val_mean_absolute_error'])
plt.title('Mean Abosulte Error')
plt.ylabel('Mean absolute error')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
pred = model.predict(X_test)
pred = pred.ravel()

# ori_test['winPlacePerc'] = np.clip(pred, a_min=0, a_max=1)

print("fix winPlacePerc")
for i in range(len(ori_test)):
    winPlacePerc = pred[i]
    maxPlace = int(ori_test.iloc[i]['maxPlace'])
    if maxPlace == 0:
        winPlacePerc = 0.0
    elif maxPlace == 1:
        winPlacePerc = 1.0
    else:
        gap = 1.0 / (maxPlace - 1)
        winPlacePerc = round(winPlacePerc / gap) * gap
    
    if winPlacePerc < 0: winPlacePerc = 0.0
    if winPlacePerc > 1: winPlacePerc = 1.0    
    pred[i] = winPlacePerc

    #if (i + 1) % 100000 == 0:
    #    print(i, flush=True, end=" ")

'''
df_test['winPlacePercPred'] = np.clip(pred, a_min=0, a_max=1)


aux = df_test.groupby(['matchId','groupId'])['winPlacePercPred'].agg('mean').groupby('matchId').rank(pct=True).reset_index()
aux.columns = ['matchId','groupId','winPlacePerc']
df_test = df_test.merge(aux, how='left', on=['matchId','groupId'])
'''
ori_test['winPlacePerc'] = pred
submission = ori_test[['Id', 'winPlacePerc']]

submission.to_csv('submission.csv', index=False)